In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir) 

In [2]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, funksvd, user_knn as uknn, item_knn as iknn 
from lenskit import topn, topnFair

In [3]:
import numpy as np
import pandas as pd
import math
import pickle
%matplotlib inline

In [4]:
test_data = pd.read_pickle("test_data_Lenskit2")
all_recs_joined = pd.read_pickle("all_recs_joined_Lenskit2")

In [5]:
testdata = all_recs_joined.loc[(all_recs_joined["user"] == 1) | (all_recs_joined["user"] == 1)]
testdata = all_recs_joined.loc[(all_recs_joined["user"] == 1) & (all_recs_joined["Algorithm"] == "ALS")]
#testdata = testdata.iloc[0:30,:] 
testdata

,item,score,user,rank,Algorithm,title,genres,(no genres listed),Action,Adventure,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
440953,285,4.912476,1,1,ALS,Beyond Bedlam (1993),Drama|Horror,0,0,0,...,0,1,0,0,0,0,0,0,0,0
440954,408,4.730659,1,2,ALS,8 Seconds (1994),Drama,0,0,0,...,0,0,0,0,0,0,0,0,0,0
440955,647,4.695571,1,3,ALS,Courage Under Fire (1996),Action|Crime|Drama|War,0,1,0,...,0,0,0,0,0,0,0,0,1,0
440956,313,4.689939,1,4,ALS,"Swan Princess, The (1994)",Animation|Children,0,0,0,...,0,0,0,0,0,0,0,0,0,0
440957,511,4.629007,1,5,ALS,"Program, The (1993)",Action|Drama,0,1,0,...,0,0,0,0,0,0,0,0,0,0
440958,513,4.600570,1,6,ALS,Radioland Murders (1994),Comedy|Mystery|Romance,0,0,0,...,0,0,0,0,1,1,0,0,0,0
440959,302,4.582783,1,7,ALS,"Queen Margot (Reine Margot, La) (1994)",Drama|Romance,0,0,0,...,0,0,0,0,0,1,0,0,0,0
440960,493,4.582732,1,8,ALS,Menace II Society (1993),Action|Crime|Drama,0,1,0,...,0,0,0,0,0,0,0,0,0,0
440961,522,4.579289,1,9,ALS,Romper Stomper (1992),Action|Drama,0,1,0,...,0,0,0,0,0,0,0,0,0,0
440962,1449,4.549330,1,10,ALS,Waiting for Guffman (1996),Comedy,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
def evaluate_genre(genre): 
    
    providers = list(all_recs_joined.iloc[:,8:])
    proItems_genre_n =  all_recs_joined.loc[ all_recs_joined[genre] == 1].item.nunique()
    print (genre)
    rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
    rla.add_metric("rND")
    #rla.add_metric("rKL")
    #rla.add_metric("rRD")
    #rla.add_metric("APCR")
    #rla.add_metric("nd_APCR")
    #rla.add_metric("equal_ex")
    #rla.add_metric("ndcg")
    results= rla.compute(testdata, test_data, genre, providers)
    #results_horror.groupby('Algorithm').mean()
    print("done" , genre)
    return results.groupby('Algorithm').mean()

In [7]:
testproviders = ["Action"]
#testproviders = list(all_recs_joined.iloc[:,8:])
allResults = []
for genre in testproviders:
    rep = evaluate_genre(genre)
    allResults.append(rep)    

Action
calculatend  10 4 1128 190
calculatend  20 6 1128 190
calculatend  30 7 1128 190
calculatend  40 9 1128 190
calculatend  50 11 1128 190
calculatend  60 12 1128 190
calculatend  70 14 1128 190
calculatend  80 15 1128 190
calculatend  90 16 1128 190
calculatend  100 18 1128 190
calculatend  110 19 1128 190
calculatend  120 23 1128 190
calculatend  130 23 1128 190
calculatend  140 25 1128 190
calculatend  150 26 1128 190
calculatend  160 27 1128 190
calculatend  170 31 1128 190
calculatend  180 32 1128 190
calculatend  190 34 1128 190
calculatend  200 35 1128 190
calculatend  210 37 1128 190
calculatend  220 38 1128 190
calculatend  230 39 1128 190
calculatend  240 41 1128 190
calculatend  250 41 1128 190
calculatend  260 42 1128 190
calculatend  270 44 1128 190
calculatend  280 46 1128 190
calculatend  290 47 1128 190
calculatend  300 48 1128 190
calculatend  310 52 1128 190
calculatend  320 52 1128 190
calculatend  330 55 1128 190
calculatend  340 55 1128 190
calculatend  350 56 

C:\Users\josse\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [8]:
output = pd.concat(allResults, keys=testproviders)
output

,,rND
,Algorithm,
Action,ALS,0.044878


In [9]:
topnFair.getNormalizer(30,7, "rND")

0.16145844796299214